In [1]:
import pandas as pd
import datetime
import utm

In [2]:
# Set holidays for 2021
holidays = []

holidays.append(datetime.datetime(2021, 1, 1))
holidays.append(datetime.datetime(2021, 1, 6))
holidays.append(datetime.datetime(2021, 3, 19))
holidays.append(datetime.datetime(2021, 4, 1))
holidays.append(datetime.datetime(2021, 4, 2))
holidays.append(datetime.datetime(2021, 5, 1))
holidays.append(datetime.datetime(2021, 5, 3))
holidays.append(datetime.datetime(2021, 10, 12))
holidays.append(datetime.datetime(2021, 11, 1))
holidays.append(datetime.datetime(2021, 12, 6))
holidays.append(datetime.datetime(2021, 12, 8))
holidays.append(datetime.datetime(2021, 12, 25))
holidays.append(datetime.datetime(2021, 5, 15))
holidays.append(datetime.datetime(2021, 11, 9))

In [3]:
# Load dataset
file_name = '2021_Accidentalidad.csv'

dt = pd.read_csv('2021_Accidentalidad.csv',sep=';', decimal=",")
dt.head()

,num_expediente,fecha,hora,localizacion,numero,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,Desconocido,Desconocido,NaN,444926.3,4470383.11,N,NaN
1,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14,444926.3,4470383.11,N,NaN
2,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,7,444926.3,4470383.11,N,NaN
3,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 10 a 14 años,Hombre,14,444926.3,4470383.11,N,NaN
4,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 35 a 39 años,Mujer,14,444926.3,4470383.11,N,NaN


In [4]:
# Transform UTM coordinates to geographic. 
# Madrid's UTM zone is 30T.
geographic = dt.apply(
    lambda row : utm.to_latlon(row['coordenada_x_utm'],
                               row['coordenada_y_utm'],
                               30,'T'),
    axis=1
).apply(pd.Series)
geographic.columns = ('lat', 'lon')
dt = pd.concat([dt, geographic], axis=1)
dt.head()

,num_expediente,fecha,hora,localizacion,numero,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,lat,lon
0,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,Desconocido,Desconocido,NaN,444926.3,4470383.11,N,NaN,40.382218,-3.648825
1,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14,444926.3,4470383.11,N,NaN,40.382218,-3.648825
2,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,7,444926.3,4470383.11,N,NaN,40.382218,-3.648825
3,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 10 a 14 años,Hombre,14,444926.3,4470383.11,N,NaN,40.382218,-3.648825
4,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 35 a 39 años,Mujer,14,444926.3,4470383.11,N,NaN,40.382218,-3.648825


In [18]:
# Generate datetime of the accident.
dt['datetime'] = pd.to_datetime(dt['fecha'] + " " + dt['hora'], format="%d/%m/%Y %H:%M:%S")
# Extract the hour of the accident.
dt['hour_interval'] = dt['datetime'].dt.hour.floordiv(4)
# Extract the weekday of the accident (Monday=0 to Sunday=6).
dt['weekday'] = dt['datetime'].dt.dayofweek
# Check whether the accident happened on a bank holiday.
dt['holidays'] = pd.to_datetime(dt['fecha'], format="%d/%m/%Y").isin(holidays)
# Check whether the accident happened on a weekend.
dt['weekend'] = dt['weekday'].isin([5,6])
# Check whether the accident happened on a workday.
dt['workdays'] = (~(dt['weekend'] | dt['holidays'])).astype(int)
dt

,num_expediente,fecha,hora,localizacion,numero,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,...,positiva_droga,lat,lon,datetime,hour,weekday,holidays,weekend,workdays,hour_interval
0,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,...,NaN,40.382218,-3.648825,2021-01-01 04:30:00,4,4,True,False,0,1
1,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,...,NaN,40.382218,-3.648825,2021-01-01 04:30:00,4,4,True,False,0,1
2,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,...,NaN,40.382218,-3.648825,2021-01-01 04:30:00,4,4,True,False,0,1
3,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Pasajero,...,NaN,40.382218,-3.648825,2021-01-01 04:30:00,4,4,True,False,0,1
4,2020S019534,01/01/2021,4:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Pasajero,...,NaN,40.382218,-3.648825,2021-01-01 04:30:00,4,4,True,False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24459,2021S015803,31/08/2021,13:50:00,"CALL. MARCELO USERA, 168",168,USERA,Alcance,Despejado,Turismo,Conductor,...,NaN,40.385444,-3.715390,2021-08-31 13:50:00,13,1,False,False,1,3
24460,2021S015803,31/08/2021,13:50:00,"CALL. MARCELO USERA, 168",168,USERA,Alcance,Despejado,Turismo,Pasajero,...,NaN,40.385444,-3.715390,2021-08-31 13:50:00,13,1,False,False,1,3
24461,2021S016898,31/08/2021,22:56:00,CALL. CAMINO DE LOS VINATEROS / CALL. CORREGID...,1,MORATALAZ,Colisión fronto-lateral,Despejado,Motocicleta hasta 125cc,Conductor,...,NaN,40.410951,-3.660492,2021-08-31 22:56:00,22,1,False,False,1,5
24462,2021S016898,31/08/2021,22:56:00,CALL. CAMINO DE LOS VINATEROS / CALL. CORREGID...,1,MORATALAZ,Colisión fronto-lateral,Despejado,Turismo,Conductor,...,NaN,40.410951,-3.660492,2021-08-31 22:56:00,22,1,False,False,1,5


In [19]:
# Save data.
idx = ['lat', 'lon', 'hour_interval', 'workdays']
dt[idx].to_csv('doc.csv', index=False ,na_rep="NULL")